# Chapter 4: Serialization and Parsing

---

## 🧠 What this chapter explores

- How Bitcoin data is **serialized** (converted to binary format) and **parsed** (read back into readable objects).
- Understanding how **little-endian and big-endian** byte orders affect Bitcoin data formats.
- Converting between Python `int` and `bytes` using endianness.
- Implementing functions to:
  - Convert bytes to integers (`little_endian_to_int`)
  - Convert integers to bytes (`int_to_little_endian`)
  - Encode and decode Bitcoin **addresses**
  - Perform **Base58 encoding and decoding**
  - Handle **checksum validation**
- Understanding the structure of Bitcoin **private keys** and their serialization in **WIF (Wallet Import Format)**.

---

## ✅ What you’ll be able to do by the end

- Write your own utility functions for serializing and deserializing Bitcoin data.
- Understand how Bitcoin addresses are derived from public keys.
- Generate and interpret Wallet Import Format (WIF) private keys.
- Begin to think like a protocol engineer: how to **safely transmit and verify** Bitcoin data.
- Lay the foundation for creating and verifying **transactions** (coming in Chapter 5).

---

## 🧰 Key Concepts and Tools

- Endianness (Little vs Big)
- `hashlib` for hashing (`sha256`, `ripemd160`)
- `base58` and `base58check`
- SEC format (public key serialization)
- WIF (private key serialization)

---


# Imports

In [5]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper

# Serialization

To communicate the classes we've created to other computers, we use serialization.

*Standard for Efficient Cryptography (SEC)* is the serialization standard. There are two SEC formats, compressed and uncompressed.

The format for uncompressed SEC for a point *P = (x.y)*:

Start wit hthe prefix byte `0x04`, append the x coordinate in 32 bytes as a **big-endian integer** then do the same for the y coordinate.

Remember, `0x` is the indicator that the format is hexadecimal.

Big-endian is the 'big end' of how a number is read. 123 = 100 + 20 + 3.

# Exercise 1



Find the uncompressed SEC format for the Public Key where the Private Key secrets are:

* 5000
* \\(2018^{5}\\)
* 0xdeadbeef12345

In [6]:
# Exercise 1 – Deriving the Public Key (Uncompressed SEC Format)

from ecc import PrivateKey, S256Point

# --- Step 1: Initialize the private key ---
# The PrivateKey class takes an integer (the secret exponent)
# and internally calculates the public key point (k * G)
e1 = PrivateKey(5000)

# --- Step 2: Print the public key point ---
# This will output an S256Point (x, y) on the secp256k1 curve
print(e1.point)

# --- Step 3: Serialize the public key in uncompressed SEC format ---
# The SEC (Standards for Efficient Cryptography) format is a binary representation of the public key.
# Uncompressed format = 0x04 + x (32 bytes) + y (32 bytes)
P1 = e1.point.sec(compressed=False)

# --- Step 4: Convert SEC binary to hex and print ---
# .hex() converts the bytes to a human-readable hex string
print(P1.hex())


S256Point(ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c, 315dc72890a4f10a1481c031b03b351b0dc79901ca18a00cf009dbdb157a1d10)
04ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c315dc72890a4f10a1481c031b03b351b0dc79901ca18a00cf009dbdb157a1d10


In [7]:
print(P1)

b'\x04\xff\xe5X\xe3\x88\x85/\x01 \xe4j\xf2\xd1\xb3p\xf8XT\xa8\xeb\x08A\x81\x1e\xce\x0e>\x03\xd2\x82\xd5|1]\xc7(\x90\xa4\xf1\n\x14\x81\xc01\xb0;5\x1b\r\xc7\x99\x01\xca\x18\xa0\x0c\xf0\t\xdb\xdb\x15z\x1d\x10'


In [8]:
e2 = PrivateKey(2018**5)
# print(e2)
print(e2.point)

P2 = e2.point.sec(compressed=False)
print(P2.hex())

S256Point(027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9d, ff87647fd023c13b4a4994f17691895806e1b40b57f4fd22581a4f46851f3b06)
04027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9dff87647fd023c13b4a4994f17691895806e1b40b57f4fd22581a4f46851f3b06


In [9]:
e3 = PrivateKey(0xdeadbeef12345)
# print(e3)
print(e3.point)

P3 = e3.point.sec(compressed=False)
print(P3)

S256Point(d90cd625ee87dd38656dd95cf79f65f60f7273b67d3096e68bd81e4f5342691f, 842efa762fd59961d0e99803c61edba8b3e3f7dc3a341836f97733aebf987121)
b'\x04\xd9\x0c\xd6%\xee\x87\xdd8em\xd9\\\xf7\x9fe\xf6\x0frs\xb6}0\x96\xe6\x8b\xd8\x1eOSBi\x1f\x84.\xfav/\xd5\x99a\xd0\xe9\x98\x03\xc6\x1e\xdb\xa8\xb3\xe3\xf7\xdc:4\x186\xf9w3\xae\xbf\x98q!'


Serialization for copmressed SEC format:

Start with the prefix byte, if *y* is even, it's `0x02`, otherwise it's `0x03`. Next, append the x coordinate in 32 bytes as a big-endian integer.

**Why?**

Evenness and oddness refer to the two possible y values (square roots) for a given x on the elliptic curve.

It's a simple way to efficiently represent the full point by keeping only x and telling whether y is even or odd.

# Exercise 2



Find the Compressed SEC format for the Public Key where the Private Key secrets are:

* 5001
* \\(2019^{5}\\)
* 0xdeadbeef54321

In [10]:
# Exercise 2

from ecc import PrivateKey

# 5001
# 2019**5
# 0xdeadbeef54321

e1 = PrivateKey(5001)
# print(e1)
print(e1.point)

P1 = e1.point.sec(compressed=True)
print(P1.hex())

S256Point(57a4f368868a8a6d572991e484e664810ff14c05c0fa023275251151fe0e53d1, 0d6cc87c5bc29b83368e17869e964f2f53d52ea3aa3e5a9efa1fa578123a0c6d)
0357a4f368868a8a6d572991e484e664810ff14c05c0fa023275251151fe0e53d1


In [11]:
from ecc import PrivateKey

e1 = PrivateKey(5001)
compressed = e1.point.sec(compressed=True)

# First byte indicates y-coordinate parity
prefix = compressed[0]

if prefix == 0x02:
    print("y is even")
elif prefix == 0x03:
    print("y is odd")
else:
    print("Unknown format")


y is odd


In [12]:
e2 = PrivateKey(2019**5)
# print(e2)
print(e2.point)

P2 = e2.point.sec(compressed=True)
print(P2.hex())

S256Point(933ec2d2b111b92737ec12f1c5d20f3233a0ad21cd8b36d0bca7a0cfa5cb8701, 96cbbfdd572f75ace44d0aa59fbab6326cb9f909385dcd066ea27affef5a488c)
02933ec2d2b111b92737ec12f1c5d20f3233a0ad21cd8b36d0bca7a0cfa5cb8701


In [13]:
e3 = PrivateKey(0xdeadbeef54321)
# print(e3)
print(e3.point)

P3 = e3.point.sec(compressed=True)
print(P3.hex())

S256Point(96be5b1292f6c856b3c5654e886fc13511462059089cdf9c479623bfcbe77690, 32555d1b027c25c2828ba96a176d78419cd1236f71558f6187aec09611325eb6)
0296be5b1292f6c856b3c5654e886fc13511462059089cdf9c479623bfcbe77690


By investigation of the serialisation, we can see they are odd, even and even.

We can also attempt to check for **parity (odd / even) by checking the y coordinate of the point:

In [14]:
from ecc import PrivateKey

e3 = PrivateKey(0xdeadbeef54321)

# Print the full point
P3 = e3.point
print(P3)

# Check if y is even or odd
y_value = P3.y.num #grabs the integer number
if y_value % 2 == 0:
    print("y is even")
else:
    print("y is odd")

# Also print compressed SEC for confirmation
print(e3.point.sec(compressed=True).hex())

S256Point(96be5b1292f6c856b3c5654e886fc13511462059089cdf9c479623bfcbe77690, 32555d1b027c25c2828ba96a176d78419cd1236f71558f6187aec09611325eb6)
y is even
0296be5b1292f6c856b3c5654e886fc13511462059089cdf9c479623bfcbe77690


**DER Signatures**

Distinguished Encoding Rules is defined by 6 formats.



# Exercise 3



Find the DER format for a signature whose `r` and `s` values are:

* r =

`0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6`

* s =

`0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec`

In [15]:
# Exercise 3

from ecc import Signature

r = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec

sig = Signature(r,s) #initialise the Signature class with (r,s).
sig.der().hex()

'3045022037206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c60221008ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec'

**0x30** or **30** — DER SEQUENCE tag  
**0x45** — Total length of the sequence: 69 bytes

**0x02** — INTEGER tag  
**0x20** — Length of r: 32 bytes

**r** = 37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6

**0x02** — INTEGER tag  
**0x21** — Length of s: 33 bytes (includes leading 0x00 to ensure positive)

**s** = 008ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec  
→ Leading **0x00** is added because the first byte of `s` is **0x8c** (≥ 0x80), which could be interpreted as negative if not padded.

**Actual s** = 8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec


**Transmitting your public key**

currently using Base58 but going to be phased out by possibly Bech32.


# Exercise 4



Convert the following hex to binary and then to Base58:

* `7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d`
* `eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c`
* `c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6`

In [16]:
# Exercise 4

from helper import encode_base58

# 7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d
# eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c
# c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6

bin1 = bytes.fromhex('7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d')
bin2 = bytes.fromhex('eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c')
bin3 = bytes.fromhex('c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6')

bin1_base58 = encode_base58(bin1)
bin2_base58 = encode_base58(bin2)
bin3_base58 = encode_base58(bin3)

print(bin1) # binary data, specifically Python bytes object` (also called a byte string).
print(bin1_base58)
print(bin2_base58)
print(bin3_base58)

b'|\x07o\xf3\x16i*=~\xb3\xc3\xbb\x0f\x8b\x14\x88\xcfr\xe1\xaf\xcd\x92\x9e)0p2\x99z\x83\x8a='
9MA8fRQrT4u8Zj8ZRd6MAiiyaxb2Y1CMpvVkHQu5hVM6
4fE3H2E6XMp4SsxtwinF7w9a34ooUrwWe4WsW1458Pd
EQJsjkd6JaGwxrjEhfeqPenqHwrBmPQZjJGNSCHBkcF7


Hex strings uses 0-9 digits and letters a-f (base-16). 

Base58 uses digits and letters but excludes confusing characters like 0, O, I, l. 

Base58 strings can look similar to the original hex if certain byte patterns are present, but will have different representations. 

**Bitcoin address Format**

mainnet address vs testnet: `0x00` and `0x6f`

5 steps to the format.

the *checksum* process is in one of the steps (4th, per the book).

# Exercise 5



Find the address corresponding to Public Keys whose Private Key secrets are:

* 5002 (use uncompressed SEC, on testnet)
* \\(2020^{5}\\) (use compressed SEC, on testnet)
* 0x12345deadbeef (use compressed SEC on mainnet)*italicized text*

____

Step 1: Initialize the private key.

Step 2: Determine the point form of the private key. 

Step 3: Use the point form to determine the address. 

ran in conda environment so we can use hashlib

In [17]:
!python --version

Python 3.11.13


In [18]:
import sys
print(sys.version)

3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]


In [19]:
import hashlib
hashlib.new('ripemd160', b'test').hexdigest()

'5e52fee47e6b070565f74372468cdc699de89107'

In [20]:
from ecc import PrivateKey

# 1. Private key: 5002 (uncompressed SEC, testnet)
key1 = PrivateKey(5002)
addr1 = key1.point.address(compressed=False, testnet=True)
print("1. Uncompressed / Testnet:", addr1)

# 2. Private key: 2020**5 (compressed SEC, testnet)
key2 = PrivateKey(2020**5)
addr2 = key2.point.address(compressed=True, testnet=True)
print("2. Compressed / Testnet:", addr2)

# 3. Private key: 0x12345deadbeef (compressed SEC, mainnet)
key3 = PrivateKey(0x12345deadbeef)
addr3 = key3.point.address(compressed=True, testnet=False)
print("3. Compressed / Mainnet:", addr3)

1. Uncompressed / Testnet: mmTPbXQFxboEtNRkwfh6K51jvdtHLxGeMA
2. Compressed / Testnet: mopVkxp8UhXqRYbCYJsbeE1h1fiF64jcoH
3. Compressed / Mainnet: 1F1Pn2y6pDb68E5nYJJeba4TLg2U7B6KF1


Real private keys are a 256 bit number. We don't need to serialise our private key as it doesnt get broadcasted. Cases where you transfer private keys from one wallet to another, we will want to serialise it. WIF key serialisation is human readable and uses the same Base58. 

WIF format creation: 

Start with a privatekey mainnet or testnet prefix, encode it in 32 byte big endian, if SEC format used for public key address is compressed, add a suffix.

Combine the prefix, serialised secret and the suffix, then hash256 to record the first 4 bytes. 

Take the the combination and the 4 bytes and encode in Base58.

# Exercise 6



Find the WIF for Private Key whose secrets are:

* 5003 (compressed, testnet)
* \\(2021^{5}\\) (uncompressed, testnet)
* 0x54321deadbeef (compressed, mainnet)

In [32]:
# Exercise 6

from ecc import PrivateKey

priv1 =  PrivateKey(5003)
priv2 =  PrivateKey(2021**5)
priv3 =  PrivateKey(0x54321deadbeef)

# addr1 = priv1.point.address(compressed = True, testnet = True)
# addr2 = priv2.point.address(compressed = False, testnet = True)
# addr3 = priv3.point.address(compressed = True, testnet = False)

priv1_wif = priv1.wif(compressed = True, testnet = True)
priv2_wif = priv2.wif(compressed = False, testnet = True)
priv3_wif = priv3.wif(compressed = True, testnet = False)

print("compressed, testnet WIF serialisation:", priv1_wif)
print("uncompressed, testnet WIF serialisation:", priv2_wif)
print("compressed, mainnet WIF serialisation:", priv3_wif)

compressed, testnet WIF serialisation: cMahea7zqjxrtgAbB7LSGbcQUr1uX1ojuat9jZodMN8rFTv2sfUK
uncompressed, testnet WIF serialisation: 91avARGdfge8E4tZfYLoxeJ5sGBdNJQH4kvjpWAxgzczjbCwxic
compressed, mainnet WIF serialisation: KwDiBf89QgGbjEhKnhXJuH7LrciVrZi3qYjgiuQJv1h8Ytr2S53a


We can now share our private key with another wallet in WIF format, which includes a checksum and optional compression flag, but it still represents the actual private key — so it must be kept secure.

WIF = structured, error-checked way to represent your private key.

It’s still the same key.

But now it:

Tells the receiver what network it’s for

Includes a compression flag (for the public key)

Makes it harder to screw up when copying

# Exercise 7



Write a function `little_endian_to_int` which takes Python bytes, interprets those bytes in Little-Endian and returns the number.

#### Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_little_endian_to_int`

In [40]:
h =bytes.fromhex('99c3980000000000')
int.from_bytes(h)

11079866634028974080

In [44]:
import importlib
import ecc
importlib.reload(ecc)

# Exercise 7

reload(helper)
run(helper.HelperTest("test_little_endian_to_int"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# Exercise 8

Write a function `int_to_little_endian` which does the reverse of the last exercise.

Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_int_to_little_endian`

In [45]:
n = 1
want = b'\x01\x00\x00\x00'

In [61]:
n.to_bytes(4)

b'\x00\x00\x00\x01'

In [68]:
n.to_bytes(4, 'little') == want

True

In [71]:
import importlib
import ecc
importlib.reload(ecc)


# Exercise 8

reload(helper)
run(helper.HelperTest("test_int_to_little_endian"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Important to remember that when converting between formats to distinguish whether little or big endian. 

# Exercise 9



Create a testnet address for yourself using a long secret that only you know. This is important as there are bots on testnet trying to steal testnet coins. Make sure you write this secret down somewhere! You will be using the secret later to sign Transactions.

In [75]:
# Exercise 9

from ecc import PrivateKey
from helper import hash256, little_endian_to_int

# select a passphrase here, add your email address into the passphrase for security
# passphrase = b'your@email.address some secret only you know'
# secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
# print an address from the public point of the private key with testnet=True

passphrase = b'the.chris.rudolph@gmail.com fiddy 4206988'

secret = little_endian_to_int(hash256(passphrase))

priv = PrivateKey(secret)

addr = priv.point.address(compressed = True, testnet = True)

print("compressed testnet public address:", addr)

compressed testnet public address: mtc5sUSUuoKzqbCAhJK7oRDVfDBmdPqr2G
